In [3]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url_amazon(search_term):
    #Generate a url from search term
    template = 'https://www.amazon.in/s?k={}&crid=1TC4WK4BF59QM&sprefix=ultrawide+monitor%2Caps%2C364&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ','+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&s=price-asc-rank&page={}'
    
    return url


def extract_record_amazon(item):
    #Extract and return data from a single record
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    try:
        # price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rank and rating
        rating = item.find('span',{'class':'a-icon-alt'}).text[0:3]
        review_count = item.find('span',{'class':'a-size-base puis-light-weight-text s-link-centralized-style'}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result


def main_amazon(search_term):
    '''Run main program routine'''
    # startup the webdriver
    driver = webdriver.Chrome()
    
    records = []
    url = get_url_amazon(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record_amazon(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # save the data to csv file
    with open('result.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f,delimiter = '|')
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [4]:
main_amazon('ultrawide monitor')